In [1]:
import pandas as pd
import panel as pn
import numpy as np
import hvplot.pandas
import folium
from panel.interact import interact
pn.extension('tabulator')
pn.extension

panel.config.panel_extension

In [2]:
df = pd.read_csv('Data/merged_data.csv', sep=',')
df.head()

,Year,Country,Region,Happiness_Rank,Happiness,Explained_by_Trust,Explained_by_GDP,Explained_by_Freedom,Explained_by_Generosity,Explained_by_Health,Explained_by_Social_support,Dystopia_Residual
0,2023,Finland,Western Europe,1,7.804,0.535,1.888,0.772,0.126,0.535,1.585,2.363
1,2023,Denmark,Western Europe,2,7.586,0.525,1.949,0.734,0.208,0.537,1.548,2.084
2,2023,Iceland,Western Europe,3,7.530,0.187,1.926,0.738,0.250,0.559,1.620,2.250
3,2023,Israel,Middle East and Northern Africa,4,7.473,0.158,1.833,0.569,0.124,0.577,1.521,2.691
4,2023,Netherlands,Western Europe,5,7.403,0.394,1.942,0.672,0.251,0.545,1.488,2.110


In [3]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('Data/merged_data.csv', sep=',')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [4]:
df.shape

(1368, 12)

In [5]:
df.isnull().sum()

Year                            0
Country                         0
Region                         67
Happiness_Rank                  0
Happiness                       1
Explained_by_Trust              2
Explained_by_GDP                1
Explained_by_Freedom            1
Explained_by_Generosity         1
Explained_by_Health             2
Explained_by_Social_support     1
Dystopia_Residual               3
dtype: int64

In [6]:
cero_happiness = df['Happiness'].isnull() == True
df.loc[cero_happiness,:]

,Year,Country,Region,Happiness_Rank,Happiness,Explained_by_Trust,Explained_by_GDP,Explained_by_Freedom,Explained_by_Generosity,Explained_by_Health,Explained_by_Social_support,Dystopia_Residual
283,2022,xx,NaN,147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1368 entries, 0 to 1367
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Year                         1368 non-null   int64  
 1   Country                      1368 non-null   object 
 2   Region                       1301 non-null   object 
 3   Happiness_Rank               1368 non-null   int64  
 4   Happiness                    1367 non-null   float64
 5   Explained_by_Trust           1366 non-null   float64
 6   Explained_by_GDP             1367 non-null   float64
 7   Explained_by_Freedom         1367 non-null   float64
 8   Explained_by_Generosity      1367 non-null   float64
 9   Explained_by_Health          1366 non-null   float64
 10  Explained_by_Social_support  1367 non-null   float64
 11  Dystopia_Residual            1365 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 128.4+ KB


In [8]:
df.nunique()

Year                              9
Country                         198
Region                           10
Happiness_Rank                  158
Happiness                      1200
Explained_by_Trust              901
Explained_by_GDP               1222
Explained_by_Freedom           1027
Explained_by_Generosity         914
Explained_by_Health            1093
Explained_by_Social_support    1166
Dystopia_Residual              1290
dtype: int64

In [9]:
df.Region.value_counts()

Region
Sub-Saharan Africa                 309
Central and Eastern Europe         249
Latin America and Caribbean        185
Western Europe                     184
Middle East and Northern Africa    155
Southeastern Asia                   80
Southern Asia                       59
Eastern Asia                        44
Australia and New Zealand           18
North America                       18
Name: count, dtype: int64

In [10]:
glob_rank = df.groupby('Country')[['Happiness_Rank','Year']].min()
glob_rank

,Happiness_Rank,Year
Country,,
Afghanistan,137,2015
Albania,83,2015
Algeria,38,2015
Angola,137,2015
Argentina,24,2015
...,...,...
Yemen,136,2015
Yemen*,132,2022
Zambia,85,2015


In [11]:
glob_hap_max = df.groupby('Year')['Happiness'].max()
glob_hap_mean = df.groupby('Year')['Happiness'].mean()
glob_hap_min = df.groupby('Year')['Happiness'].min() 

In [12]:
country_happiness = df.groupby('Country')[['Year','Happiness','Happiness_Rank']]

In [13]:
rank_1 = df['Happiness_Rank'] == 1
df.loc[rank_1,:]

,Year,Country,Region,Happiness_Rank,Happiness,Explained_by_Trust,Explained_by_GDP,Explained_by_Freedom,Explained_by_Generosity,Explained_by_Health,Explained_by_Social_support,Dystopia_Residual
0,2023,Finland,Western Europe,1,7.8040,0.535000,1.888000,0.772000,0.126000,0.535000,1.585000,2.363000
137,2022,Finland,Western Europe,1,7.8210,0.534000,1.892000,0.736000,0.109000,0.775000,1.258000,2.518000
284,2021,Finland,Western Europe,1,7.8420,0.481000,1.446000,0.691000,0.124000,0.741000,1.106000,3.253000
433,2020,Finland,Western Europe,1,7.8087,0.477857,1.285190,0.662317,0.159670,0.961271,1.499526,2.762835
586,2019,Finland,Western Europe,1,7.7690,0.393000,1.340000,0.596000,0.153000,0.986000,1.587000,2.714000
742,2018,Finland,Western Europe,1,7.6320,0.393000,1.305000,0.681000,0.202000,0.874000,1.592000,2.585000
898,2017,Norway,Western Europe,1,7.5370,0.315964,1.616463,0.635423,0.362012,0.796667,1.533524,2.277027
1053,2016,Denmark,Western Europe,1,7.5260,0.444530,1.441780,0.579410,0.361710,0.795040,1.163740,2.739390
1210,2015,Switzerland,Western Europe,1,7.5870,0.419780,1.396510,0.665570,0.296780,0.941430,1.349510,2.517380


In [14]:
rank_2 = df['Happiness_Rank'] == 2
df.loc[rank_2,:]

,Year,Country,Region,Happiness_Rank,Happiness,Explained_by_Trust,Explained_by_GDP,Explained_by_Freedom,Explained_by_Generosity,Explained_by_Health,Explained_by_Social_support,Dystopia_Residual
1,2023,Denmark,Western Europe,2,7.5860,0.52500,1.949000,0.734000,0.208000,0.537000,1.548000,2.084000
138,2022,Denmark,Western Europe,2,7.6360,0.53200,1.953000,0.719000,0.188000,0.777000,1.243000,2.226000
285,2021,Denmark,Western Europe,2,7.6200,0.48500,1.502000,0.686000,0.208000,0.763000,1.108000,2.868000
434,2020,Denmark,Western Europe,2,7.6456,0.49526,1.326949,0.665040,0.242793,0.979333,1.503449,2.432741
587,2019,Denmark,Western Europe,2,7.6000,0.41000,1.383000,0.592000,0.252000,0.996000,1.573000,2.394000
743,2018,Norway,Western Europe,2,7.5940,0.34000,1.456000,0.686000,0.286000,0.861000,1.582000,2.383000
899,2017,Denmark,Western Europe,2,7.5220,0.40077,1.482383,0.626007,0.355280,0.792566,1.551122,2.313707
1054,2016,Switzerland,Western Europe,2,7.5090,0.41203,1.527330,0.585570,0.280830,0.863030,1.145240,2.694630
1211,2015,Iceland,Western Europe,2,7.5610,0.14145,1.302320,0.628770,0.436300,0.947840,1.402230,2.702010


In [15]:
df['Happiness_Rank'].max()

158

In [16]:
df['Happiness'].min()

1.859

# Make df interactive

In [17]:
idf = df.interactive()

In [18]:
idf.head(1)

# Define Panel Widgets 

In [19]:
from bokeh.models.formatters import PrintfTickFormatter

rank_slider = pn.widgets.IntSlider(name='Happiness Rank', start=1, end=158, step=1, value=1)
rank_slider

IntSlider(end=158, name='Happiness Rank', start=1, value=1)

In [67]:
# Radio buttons for CO2 measures
yaxis_Happiness = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Happiness'],
    button_type='success'
)

In [75]:
year = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022','2023']

happiness_pipeline = (
    idf[
        (idf.Happiness_Rank <= rank_slider) #&
        #(idf.Year.isin(year))
    ]
    .groupby(['Country', 'Year']).min()
    #.to_frame()
    .reset_index()
    .sort_values(by='Happiness_Rank', ascending=True)  
    #.reset_index(drop=True)
)

In [78]:
happiness_pipeline[-10:]

In [70]:
happiness_plot = happiness_pipeline.hvplot(x='Year', by='Country', y=yaxis_Happiness, line_width=1.5, title="test", grid=True, hover_cols='Happiness_Rank')
happiness_plot

C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


In [41]:
rank_slider.value

10

In [91]:
rank = idf['Happiness_Rank'] == rank_slider.value
#no_1_2015_2023 = 
idf.loc[rank,:].sort_values(by='Year', ascending=True)


In [125]:
year = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022','2023']

happiness_pipeline_test = (
    idf[
        (idf.Happiness_Rank <= rank_slider) #&
        #(idf.Year.isin(year))
    ]
    .sort_values(by='Year', ascending=True)
    #.groupby(['Country', 'Year']).min()
    #.to_frame()
    #.reset_index()
    #.sort_values(by='Happiness_Rank', ascending=True)  
    #.reset_index(drop=True)
)

#rank = idf['Happiness_Rank'] == rank_slider.value
#no_1_2015_2023 = 
#idf.loc[rank,:].sort_values(by='Year', ascending=True)
happiness_pipeline_test

In [131]:
yaxis_Happiness_test = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Happiness'],
    button_type='success'
)

In [138]:
happiness_plot_test = happiness_pipeline_test.hvplot.line(x='Year', 
                                                     by='Country', 
                                                     y=yaxis_Happiness_test, 
                                                     line_width=1.5, title="test", 
                                                     grid=True, 
                                                     hover_cols='Happiness_Rank',
                                                     cmap='Dark2',
                                                         width=800,
                                                         height=400)
happiness_plot_test

C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
